In [1]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial import distance
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors


# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read in CSV
df = pd.read_csv("Resources/spotify_tracks.csv", encoding='latin1')
df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [3]:
# Check for null values and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  liveness          11

In [4]:
# View the metadata for each column
df.describe()

,Unnamed: 0,popularity,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,114000.000000,114000.000000,1.140000e+05,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000
mean,56999.500000,33.238535,2.280292e+05,0.566800,0.641383,5.309140,-8.258960,0.637553,0.084652,0.314910,0.156050,0.213553,0.474068,122.147837,3.904035
std,32909.109681,22.305078,1.072977e+05,0.173542,0.251529,3.559987,5.029337,0.480709,0.105732,0.332523,0.309555,0.190378,0.259261,29.978197,0.432621
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,-49.531000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28499.750000,17.000000,1.740660e+05,0.456000,0.472000,2.000000,-10.013000,0.000000,0.035900,0.016900,0.000000,0.098000,0.260000,99.218750,4.000000
50%,56999.500000,35.000000,2.129060e+05,0.580000,0.685000,5.000000,-7.004000,1.000000,0.048900,0.169000,0.000042,0.132000,0.464000,122.017000,4.000000
75%,85499.250000,50.000000,2.615060e+05,0.695000,0.854000,8.000000,-5.003000,1.000000,0.084500,0.598000,0.049000,0.273000,0.683000,140.071000,4.000000
max,113999.000000,100.000000,5.237295e+06,0.985000,1.000000,11.000000,4.532000,1.000000,0.965000,0.996000,1.000000,1.000000,0.995000,243.372000,5.000000


In [5]:
null_rows = df.loc[df['album_name'].isnull()]
print(null_rows)

       Unnamed: 0                track_id artists album_name track_name  \
65900       65900  1kR4gIb7nGxHPI3D2ifs59     NaN        NaN        NaN   

       popularity  duration_ms  explicit  danceability  energy  key  loudness  \
65900           0            0     False         0.501   0.583    7     -9.46   

       mode  speechiness  acousticness  instrumentalness  liveness  valence  \
65900     0       0.0605          0.69           0.00396    0.0747    0.734   

         tempo  time_signature track_genre  
65900  138.391               4       k-pop  


In [6]:
df = df.dropna(subset=['album_name'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113999 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        113999 non-null  int64  
 1   track_id          113999 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        113999 non-null  int64  
 6   duration_ms       113999 non-null  int64  
 7   explicit          113999 non-null  bool   
 8   danceability      113999 non-null  float64
 9   energy            113999 non-null  float64
 10  key               113999 non-null  int64  
 11  loudness          113999 non-null  float64
 12  mode              113999 non-null  int64  
 13  speechiness       113999 non-null  float64
 14  acousticness      113999 non-null  float64
 15  instrumentalness  113999 non-null  float64
 16  liveness          113999 

In [8]:
df.track_id.nunique()

89740

In [7]:
df.drop_duplicates(subset=['track_id'], keep='first', inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89740 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        89740 non-null  int64  
 1   track_id          89740 non-null  object 
 2   artists           89740 non-null  object 
 3   album_name        89740 non-null  object 
 4   track_name        89740 non-null  object 
 5   popularity        89740 non-null  int64  
 6   duration_ms       89740 non-null  int64  
 7   explicit          89740 non-null  bool   
 8   danceability      89740 non-null  float64
 9   energy            89740 non-null  float64
 10  key               89740 non-null  int64  
 11  loudness          89740 non-null  float64
 12  mode              89740 non-null  int64  
 13  speechiness       89740 non-null  float64
 14  acousticness      89740 non-null  float64
 15  instrumentalness  89740 non-null  float64
 16  liveness          89740 non-null  float64
 1

In [10]:
df.to_csv('Resources/clean_spotify.csv', index=False)

In [112]:
df.track_genre.nunique()
# 

113

In [8]:
genre_df = df.groupby('track_genre')['track_id'].nunique().reset_index(name='song_count')
print(genre_df)

     track_genre  song_count
0       acoustic        1000
1       afrobeat         999
2       alt-rock         999
3    alternative         407
4        ambient         999
..           ...         ...
108       techno         416
109       trance         708
110     trip-hop         904
111      turkish         870
112  world-music         923

[113 rows x 2 columns]


In [9]:
genre_df = genre_df.sort_values(by='song_count', ascending=True)
print(genre_df)

   track_genre  song_count
89   reggaeton          74
56       indie         134
53       house         210
85        punk         226
71       metal         232
..         ...         ...
4      ambient         999
2     alt-rock         999
1     afrobeat         999
12    cantopop         999
0     acoustic        1000

[113 rows x 2 columns]


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89740 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        89740 non-null  int64  
 1   track_id          89740 non-null  object 
 2   artists           89740 non-null  object 
 3   album_name        89740 non-null  object 
 4   track_name        89740 non-null  object 
 5   popularity        89740 non-null  int64  
 6   duration_ms       89740 non-null  int64  
 7   explicit          89740 non-null  bool   
 8   danceability      89740 non-null  float64
 9   energy            89740 non-null  float64
 10  key               89740 non-null  int64  
 11  loudness          89740 non-null  float64
 12  mode              89740 non-null  int64  
 13  speechiness       89740 non-null  float64
 14  acousticness      89740 non-null  float64
 15  instrumentalness  89740 non-null  float64
 16  liveness          89740 non-null  float64
 1

In [26]:
numeric_columns = ['popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

numeric_df = df.loc[:, numeric_columns]
numeric_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89740 entries, 0 to 113999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   popularity        89740 non-null  int64  
 1   duration_ms       89740 non-null  int64  
 2   explicit          89740 non-null  bool   
 3   danceability      89740 non-null  float64
 4   energy            89740 non-null  float64
 5   key               89740 non-null  int64  
 6   loudness          89740 non-null  float64
 7   mode              89740 non-null  int64  
 8   speechiness       89740 non-null  float64
 9   acousticness      89740 non-null  float64
 10  instrumentalness  89740 non-null  float64
 11  liveness          89740 non-null  float64
 12  valence           89740 non-null  float64
 13  tempo             89740 non-null  float64
 14  time_signature    89740 non-null  int64  
dtypes: bool(1), float64(9), int64(5)
memory usage: 10.4 MB


In [34]:
data_types_counts = df.dtypes.value_counts()
print(data_types_counts)

float64    10
int64       6
object      5
Name: count, dtype: int64


In [27]:
numeric_df['explicit'] = numeric_df['explicit'].astype(float)

In [28]:
# Assuming 'df' is your DataFrame
int_columns = numeric_df.select_dtypes(include=['int64']).columns
numeric_df[int_columns] = numeric_df[int_columns].astype('float64')

# Verify the data types after conversion
print(numeric_df.dtypes)

popularity          float64
duration_ms         float64
explicit            float64
danceability        float64
energy              float64
key                 float64
loudness            float64
mode                float64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature      float64
dtype: object


In [29]:
# initialize
scaler = StandardScaler()

# fit
scaler.fit(numeric_df)

# predict/transform
scaled_data = scaler.transform(numeric_df)
df_scaled = pd.DataFrame(scaled_data, columns=numeric_columns, index = df.track_id)

df_scaled.head()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
track_id,,,,,,,,,,,,,,,
5SuOikwiRyPMVoIQDJUgSV,1.933925,0.013472,-0.306447,0.644253,-0.675975,-1.203275,0.335727,-1.324621,0.490458,-0.875166,-0.535482,0.723656,0.934047,-1.133599,0.226216
4qPNDBW1i3p13qLCt0Ki3A,1.059312,-0.704186,-0.306447,-0.804604,-1.825602,-1.203275,-1.673087,0.754933,-0.098364,1.760810,-0.535468,-0.595078,-0.770269,-1.479843,0.226216
1iJBSr7s7jYXzM8EGcbK5b,1.156491,-0.162188,-0.306447,-0.702731,-1.073473,-1.484183,-0.236524,0.754933,-0.280219,-0.349626,-0.535485,-0.512978,-1.329497,-1.518259,0.226216
6lfxq3CG4xtTiEg7opyCyx,1.836746,-0.240925,-0.306447,-1.676182,-2.240247,-1.484183,-1.918228,0.754933,-0.451480,1.704650,-0.535266,-0.436009,-1.241999,1.981635,-1.979174
5vjLSffimiIP26QG5WcN2K,2.371232,-0.268195,-0.306447,0.315996,-0.746122,-0.922368,-0.226373,0.754933,-0.307585,0.415925,-0.535485,-0.687954,-1.150696,-0.070030,0.226216


In [30]:
string_columns = ['track_id', 'track_name', 'album_name', 'artists' , 'track_genre']
string_df = df.loc[:, string_columns]
string_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89740 entries, 0 to 113999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   track_id     89740 non-null  object
 1   track_name   89740 non-null  object
 2   album_name   89740 non-null  object
 3   artists      89740 non-null  object
 4   track_genre  89740 non-null  object
dtypes: object(5)
memory usage: 4.1+ MB


In [31]:
track_data = df.loc[df['track_name'] == 'How to Save a Life']

# Print all columns for the specified track_name value
print(track_data)

       Unnamed: 0                track_id   artists          album_name  \
79053       79053  5fVZC9GiM4e8vu99W0Xf6J  The Fray  How To Save A Life   

               track_name  popularity  duration_ms  explicit  danceability  \
79053  How to Save a Life          80       262533     False          0.64   

       energy  key  loudness  mode  speechiness  acousticness  \
79053   0.743   10     -4.08     1       0.0379         0.269   

       instrumentalness  liveness  valence    tempo  time_signature  \
79053               0.0     0.101    0.361  122.035               4   

      track_genre  
79053       piano  


In [32]:
numeric_columns

['popularity',
 'duration_ms',
 'explicit',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature']

In [33]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()

numeric_col = df.select_dtypes(include=np.number).columns
data_df = pd.DataFrame(scaler.fit_transform(df[numeric_columns]), columns=numeric_columns, index=df['track_id'])

data_df.head()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
track_id,,,,,,,,,,,,,,,
5SuOikwiRyPMVoIQDJUgSV,0.73,0.042473,0.0,0.686294,0.4610,0.090909,0.791392,0.0,0.148187,0.032329,0.000001,0.3580,0.718593,0.361245,0.8
4qPNDBW1i3p13qLCt0Ki3A,0.55,0.026971,0.0,0.426396,0.1660,0.090909,0.597377,1.0,0.079067,0.927711,0.000006,0.1010,0.268342,0.318397,0.8
1iJBSr7s7jYXzM8EGcbK5b,0.57,0.038679,0.0,0.444670,0.3590,0.000000,0.736123,1.0,0.057720,0.210843,0.000000,0.1170,0.120603,0.313643,0.8
6lfxq3CG4xtTiEg7opyCyx,0.71,0.036978,0.0,0.270051,0.0596,0.000000,0.573701,1.0,0.037617,0.908635,0.000071,0.1320,0.143719,0.746758,0.6
5vjLSffimiIP26QG5WcN2K,0.82,0.036389,0.0,0.627411,0.4430,0.181818,0.737103,1.0,0.054508,0.470884,0.000000,0.0829,0.167839,0.492863,0.8


In [107]:
# Check to see if song is in dataset
song= df[(df['track_name'] == 'See You Again')]
song

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
716,3XfMyT4Xf5LegDhvbrFEjp,Boyce Avenue;Bea Miller,"Cover Sessions, Vol. 4",See You Again,50,239802,0.0,0.570,0.413,9,-7.034,1,0.0345,0.5350,0.000000,0.0760,0.2780,152.032,4,acoustic
14329,7qdNE9DyIKk3invtcxdGG8,One Voice Children's Choir,Memories,See You Again,43,209881,0.0,0.640,0.406,10,-9.337,1,0.0476,0.7870,0.000000,0.0725,0.2470,80.029,4,children
20696,62lmjlPu5Vwd3h18FMSz1G,Wiz Khalifa;Charlie Puth,Give You Love - Cozy Hits,See You Again,2,229525,0.0,0.690,0.480,10,-7.503,1,0.0816,0.3690,0.000000,0.0649,0.2860,80.025,4,dance
20697,0FtOxBrDP67usYNbqOuy7T,Wiz Khalifa;Charlie Puth,On Chill - Rap & RnB,See You Again,0,229525,0.0,0.690,0.480,10,-7.503,1,0.0816,0.3690,0.000000,0.0649,0.2860,80.025,4,dance
29795,4pXG8Q82L8WvypAm5Wo86y,Seven Lions;Jason Ross;Fiora,See You Again EP,See You Again,46,263200,0.0,0.248,0.588,8,-6.292,1,0.0459,0.0415,0.000033,0.1230,0.0382,150.419,4,dubstep


In [34]:
# 
trackNameListened = "2055"
track_id = df[(df['track_name'] == trackNameListened)][['track_id']]
track_id = track_id.values[0][0]

target_track = list(data_df.loc[track_id])

In [36]:
data_result = pd.DataFrame()
data_result['euclidean'] = [distance.euclidean(obj, target_track) for index, obj in data_df.iterrows()]
data_result['track_id'] = data_df.index
data_result.head()

,euclidean,track_id
0,1.232549,5SuOikwiRyPMVoIQDJUgSV
1,1.717971,4qPNDBW1i3p13qLCt0Ki3A
2,1.706314,1iJBSr7s7jYXzM8EGcbK5b
3,1.884365,6lfxq3CG4xtTiEg7opyCyx
4,1.568378,5vjLSffimiIP26QG5WcN2K


In [37]:
data_rec = data_result.sort_values(by=['euclidean']).iloc[:6]

In [38]:
data_init = df.set_index(df.loc[:, 'track_id'])
track_list = pd.DataFrame()
for i in list(data_rec.loc[:, 'track_id']):
    if i in list(df.loc[:, 'track_id']):
        track_info = data_init.loc[[i], ['track_name', 'artists']]
        track_list = pd.concat([track_list, track_info], ignore_index=True)

In [39]:
recomended = track_list.values.tolist()
print(f"""You've just listened:  \n \t - {recomended[0][0]} - {recomended[0][1]} 
Now you might want to listen to : 
\n \t - '{recomended[1][0]} - {recomended[1][1]}'
Or maybe any of these:
\n \t - '{recomended[2][0]} - {recomended[2][1]}' 
\n \t - '{recomended[3][0]} - {recomended[3][1]}'
\n \t - '{recomended[4][0]} - {recomended[4][1]}'
\n \t - '{recomended[5][0]} - {recomended[5][1]}'  """)

You've just listened:  
 	 - 2055 - Sleepy Hallow 
Now you might want to listen to : 

 	 - 'Pain - Josh A'
Or maybe any of these:

 	 - 'She's So Nice - Pink Guy' 

 	 - 'Peru - Remix - Fireboy DML;21 Savage;Blxst'

 	 - 'Space Cadet (feat. Gunna) - Metro Boomin;Gunna'

 	 - 'A Gangster's Wife - Ms Krazie;Chino Grande'  


In [40]:
df_model = data_df.reset_index()
df_model.head()

,track_id,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,5SuOikwiRyPMVoIQDJUgSV,0.73,0.042473,0.0,0.686294,0.4610,0.090909,0.791392,0.0,0.148187,0.032329,0.000001,0.3580,0.718593,0.361245,0.8
1,4qPNDBW1i3p13qLCt0Ki3A,0.55,0.026971,0.0,0.426396,0.1660,0.090909,0.597377,1.0,0.079067,0.927711,0.000006,0.1010,0.268342,0.318397,0.8
2,1iJBSr7s7jYXzM8EGcbK5b,0.57,0.038679,0.0,0.444670,0.3590,0.000000,0.736123,1.0,0.057720,0.210843,0.000000,0.1170,0.120603,0.313643,0.8
3,6lfxq3CG4xtTiEg7opyCyx,0.71,0.036978,0.0,0.270051,0.0596,0.000000,0.573701,1.0,0.037617,0.908635,0.000071,0.1320,0.143719,0.746758,0.6
4,5vjLSffimiIP26QG5WcN2K,0.82,0.036389,0.0,0.627411,0.4430,0.181818,0.737103,1.0,0.054508,0.470884,0.000000,0.0829,0.167839,0.492863,0.8


In [51]:
genre_df = pd.get_dummies(df, 
                        columns = ['track_genre'], 
                        prefix = ['track_genre'])
genre_df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre_acoustic,track_genre_afrobeat,track_genre_alt-rock,track_genre_alternative,track_genre_ambient,track_genre_anime,track_genre_black-metal,track_genre_bluegrass,track_genre_blues,track_genre_brazil,track_genre_breakbeat,track_genre_british,track_genre_cantopop,track_genre_chicago-house,track_genre_children,track_genre_chill,track_genre_classical,track_genre_club,track_genre_comedy,track_genre_country,track_genre_dance,track_genre_dancehall,track_genre_death-metal,track_genre_deep-house,track_genre_detroit-techno,track_genre_disco,track_genre_disney,track_genre_drum-and-bass,track_genre_dub,track_genre_dubstep,track_genre_edm,track_genre_electro,track_genre_electronic,track_genre_emo,track_genre_folk,track_genre_forro,track_genre_french,track_genre_funk,track_genre_garage,track_genre_german,track_genre_gospel,track_genre_goth,track_genre_grindcore,track_genre_groove,track_genre_grunge,track_genre_guitar,track_genre_happy,track_genre_hard-rock,track_genre_hardcore,track_genre_hardstyle,track_genre_heavy-metal,track_genre_hip-hop,track_genre_honky-tonk,track_genre_house,track_genre_idm,track_genre_indian,track_genre_indie,track_genre_indie-pop,track_genre_industrial,track_genre_iranian,track_genre_j-dance,track_genre_j-idol,track_genre_j-pop,track_genre_j-rock,track_genre_jazz,track_genre_k-pop,track_genre_kids,track_genre_latin,track_genre_latino,track_genre_malay,track_genre_mandopop,track_genre_metal,track_genre_metalcore,track_genre_minimal-techno,track_genre_mpb,track_genre_new-age,track_genre_opera,track_genre_pagode,track_genre_party,track_genre_piano,track_genre_pop,track_genre_pop-film,track_genre_power-pop,track_genre_progressive-house,track_genre_psych-rock,track_genre_punk,track_genre_punk-rock,track_genre_r-n-b,track_genre_reggae,track_genre_reggaeton,track_genre_rock,track_genre_rock-n-roll,track_genre_rockabilly,track_genre_romance,track_genre_sad,track_genre_salsa,track_genre_samba,track_genre_sertanejo,track_genre_show-tunes,track_genre_singer-songwriter,track_genre_ska,track_genre_sleep,track_genre_soul,track_genre_spanish,track_genre_study,track_genre_swedish,track_genre_synth-pop,track_genre_tango,track_genre_techno,track_genre_trance,track_genre_trip-hop,track_genre_turkish,track_genre_world-music
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa

In [42]:
# genre_df.drop(columns=['Unnamed: 0'], inplace=True)
genre_df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre_acoustic,track_genre_afrobeat,track_genre_alt-rock,track_genre_alternative,track_genre_ambient,track_genre_anime,track_genre_black-metal,track_genre_bluegrass,track_genre_blues,track_genre_brazil,track_genre_breakbeat,track_genre_british,track_genre_cantopop,track_genre_chicago-house,track_genre_children,track_genre_chill,track_genre_classical,track_genre_club,track_genre_comedy,track_genre_country,track_genre_dance,track_genre_dancehall,track_genre_death-metal,track_genre_deep-house,track_genre_detroit-techno,track_genre_disco,track_genre_disney,track_genre_drum-and-bass,track_genre_dub,track_genre_dubstep,track_genre_edm,track_genre_electro,track_genre_electronic,track_genre_emo,track_genre_folk,track_genre_forro,track_genre_french,track_genre_funk,track_genre_garage,track_genre_german,track_genre_gospel,track_genre_goth,track_genre_grindcore,track_genre_groove,track_genre_grunge,track_genre_guitar,track_genre_happy,track_genre_hard-rock,track_genre_hardcore,track_genre_hardstyle,track_genre_heavy-metal,track_genre_hip-hop,track_genre_honky-tonk,track_genre_house,track_genre_idm,track_genre_indian,track_genre_indie,track_genre_indie-pop,track_genre_industrial,track_genre_iranian,track_genre_j-dance,track_genre_j-idol,track_genre_j-pop,track_genre_j-rock,track_genre_jazz,track_genre_k-pop,track_genre_kids,track_genre_latin,track_genre_latino,track_genre_malay,track_genre_mandopop,track_genre_metal,track_genre_metalcore,track_genre_minimal-techno,track_genre_mpb,track_genre_new-age,track_genre_opera,track_genre_pagode,track_genre_party,track_genre_piano,track_genre_pop,track_genre_pop-film,track_genre_power-pop,track_genre_progressive-house,track_genre_psych-rock,track_genre_punk,track_genre_punk-rock,track_genre_r-n-b,track_genre_reggae,track_genre_reggaeton,track_genre_rock,track_genre_rock-n-roll,track_genre_rockabilly,track_genre_romance,track_genre_sad,track_genre_salsa,track_genre_samba,track_genre_sertanejo,track_genre_show-tunes,track_genre_singer-songwriter,track_genre_ska,track_genre_sleep,track_genre_soul,track_genre_spanish,track_genre_study,track_genre_swedish,track_genre_synth-pop,track_genre_tango,track_genre_techno,track_genre_trance,track_genre_trip-hop,track_genre_turkish,track_genre_world-music
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa

In [24]:
df.columns

Index(['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name',
       'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'track_genre'],
      dtype='object')

In [52]:
genre_df.drop(columns=['artists', 'album_name', 'track_name', 'popularity',
       'duration_ms', 'explicit', 'danceability', 'energy', 'key','loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'], inplace=True)
genre_df.head()

,Unnamed: 0,track_id,track_genre_acoustic,track_genre_afrobeat,track_genre_alt-rock,track_genre_alternative,track_genre_ambient,track_genre_anime,track_genre_black-metal,track_genre_bluegrass,track_genre_blues,track_genre_brazil,track_genre_breakbeat,track_genre_british,track_genre_cantopop,track_genre_chicago-house,track_genre_children,track_genre_chill,track_genre_classical,track_genre_club,track_genre_comedy,track_genre_country,track_genre_dance,track_genre_dancehall,track_genre_death-metal,track_genre_deep-house,track_genre_detroit-techno,track_genre_disco,track_genre_disney,track_genre_drum-and-bass,track_genre_dub,track_genre_dubstep,track_genre_edm,track_genre_electro,track_genre_electronic,track_genre_emo,track_genre_folk,track_genre_forro,track_genre_french,track_genre_funk,track_genre_garage,track_genre_german,track_genre_gospel,track_genre_goth,track_genre_grindcore,track_genre_groove,track_genre_grunge,track_genre_guitar,track_genre_happy,track_genre_hard-rock,track_genre_hardcore,track_genre_hardstyle,track_genre_heavy-metal,track_genre_hip-hop,track_genre_honky-tonk,track_genre_house,track_genre_idm,track_genre_indian,track_genre_indie,track_genre_indie-pop,track_genre_industrial,track_genre_iranian,track_genre_j-dance,track_genre_j-idol,track_genre_j-pop,track_genre_j-rock,track_genre_jazz,track_genre_k-pop,track_genre_kids,track_genre_latin,track_genre_latino,track_genre_malay,track_genre_mandopop,track_genre_metal,track_genre_metalcore,track_genre_minimal-techno,track_genre_mpb,track_genre_new-age,track_genre_opera,track_genre_pagode,track_genre_party,track_genre_piano,track_genre_pop,track_genre_pop-film,track_genre_power-pop,track_genre_progressive-house,track_genre_psych-rock,track_genre_punk,track_genre_punk-rock,track_genre_r-n-b,track_genre_reggae,track_genre_reggaeton,track_genre_rock,track_genre_rock-n-roll,track_genre_rockabilly,track_genre_romance,track_genre_sad,track_genre_salsa,track_genre_samba,track_genre_sertanejo,track_genre_show-tunes,track_genre_singer-songwriter,track_genre_ska,track_genre_sleep,track_genre_soul,track_genre_spanish,track_genre_study,track_genre_swedish,track_genre_synth-pop,track_genre_tango,track_genre_techno,track_genre_trance,track_genre_trip-hop,track_genre_turkish,track_genre_world-music
0,0,5SuOikwiRyPMVoIQDJUgSV,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,1,4qPNDBW1i3p13qLCt0Ki3A,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2,1iJBSr7s7jYXzM8EGcbK5b,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

In [53]:
genre_df.columns

Index(['Unnamed: 0', 'track_id', 'track_genre_acoustic',
       'track_genre_afrobeat', 'track_genre_alt-rock',
       'track_genre_alternative', 'track_genre_ambient', 'track_genre_anime',
       'track_genre_black-metal', 'track_genre_bluegrass',
       ...
       'track_genre_spanish', 'track_genre_study', 'track_genre_swedish',
       'track_genre_synth-pop', 'track_genre_tango', 'track_genre_techno',
       'track_genre_trance', 'track_genre_trip-hop', 'track_genre_turkish',
       'track_genre_world-music'],
      dtype='object', length=115)

In [54]:
genre_features = genre_df.columns.tolist()
genre_features

['Unnamed: 0',
 'track_id',
 'track_genre_acoustic',
 'track_genre_afrobeat',
 'track_genre_alt-rock',
 'track_genre_alternative',
 'track_genre_ambient',
 'track_genre_anime',
 'track_genre_black-metal',
 'track_genre_bluegrass',
 'track_genre_blues',
 'track_genre_brazil',
 'track_genre_breakbeat',
 'track_genre_british',
 'track_genre_cantopop',
 'track_genre_chicago-house',
 'track_genre_children',
 'track_genre_chill',
 'track_genre_classical',
 'track_genre_club',
 'track_genre_comedy',
 'track_genre_country',
 'track_genre_dance',
 'track_genre_dancehall',
 'track_genre_death-metal',
 'track_genre_deep-house',
 'track_genre_detroit-techno',
 'track_genre_disco',
 'track_genre_disney',
 'track_genre_drum-and-bass',
 'track_genre_dub',
 'track_genre_dubstep',
 'track_genre_edm',
 'track_genre_electro',
 'track_genre_electronic',
 'track_genre_emo',
 'track_genre_folk',
 'track_genre_forro',
 'track_genre_french',
 'track_genre_funk',
 'track_genre_garage',
 'track_genre_german',
 

In [55]:
numeric_columns

['popularity',
 'duration_ms',
 'explicit',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature']

In [57]:
features = genre_features + numeric_columns

In [56]:
song_df = df[['track_id', 'track_name', 'artists', 'album_name']]
df_model = pd.merge(df_model, song_df, on='track_id', how='left')
df_model.head()

,track_id,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_name_x,artists_x,album_name_x,track_name_y,artists_y,album_name_y
0,5SuOikwiRyPMVoIQDJUgSV,0.73,0.042473,0.0,0.686294,0.4610,0.090909,0.791392,0.0,0.148187,0.032329,0.000001,0.3580,0.718593,0.361245,0.8,Comedy,Gen Hoshino,Comedy,Comedy,Gen Hoshino,Comedy
1,4qPNDBW1i3p13qLCt0Ki3A,0.55,0.026971,0.0,0.426396,0.1660,0.090909,0.597377,1.0,0.079067,0.927711,0.000006,0.1010,0.268342,0.318397,0.8,Ghost - Acoustic,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,Ben Woodward,Ghost (Acoustic)
2,1iJBSr7s7jYXzM8EGcbK5b,0.57,0.038679,0.0,0.444670,0.3590,0.000000,0.736123,1.0,0.057720,0.210843,0.000000,0.1170,0.120603,0.313643,0.8,To Begin Again,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,Ingrid Michaelson;ZAYN,To Begin Again
3,6lfxq3CG4xtTiEg7opyCyx,0.71,0.036978,0.0,0.270051,0.0596,0.000000,0.573701,1.0,0.037617,0.908635,0.000071,0.1320,0.143719,0.746758,0.6,Can't Help Falling In Love,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...
4,5vjLSffimiIP26QG5WcN2K,0.82,0.036389,0.0,0.627411,0.4430,0.181818,0.737103,1.0,0.054508,0.470884,0.000000,0.0829,0.167839,0.492863,0.8,Hold On,Chord Overstreet,Hold On,Hold On,Chord Overstreet,Hold On


In [64]:
df_final = pd.merge(df_model, genre_df, on='track_id', how='left')
df_final.head()

,track_id,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_name_x,artists_x,album_name_x,track_name_y,artists_y,album_name_y,Unnamed: 0,track_genre_acoustic,track_genre_afrobeat,track_genre_alt-rock,track_genre_alternative,track_genre_ambient,track_genre_anime,track_genre_black-metal,track_genre_bluegrass,track_genre_blues,track_genre_brazil,track_genre_breakbeat,track_genre_british,track_genre_cantopop,track_genre_chicago-house,track_genre_children,track_genre_chill,track_genre_classical,track_genre_club,track_genre_comedy,track_genre_country,track_genre_dance,track_genre_dancehall,track_genre_death-metal,track_genre_deep-house,track_genre_detroit-techno,track_genre_disco,track_genre_disney,track_genre_drum-and-bass,track_genre_dub,track_genre_dubstep,track_genre_edm,track_genre_electro,track_genre_electronic,track_genre_emo,track_genre_folk,track_genre_forro,track_genre_french,track_genre_funk,track_genre_garage,track_genre_german,track_genre_gospel,track_genre_goth,track_genre_grindcore,track_genre_groove,track_genre_grunge,track_genre_guitar,track_genre_happy,track_genre_hard-rock,track_genre_hardcore,track_genre_hardstyle,track_genre_heavy-metal,track_genre_hip-hop,track_genre_honky-tonk,track_genre_house,track_genre_idm,track_genre_indian,track_genre_indie,track_genre_indie-pop,track_genre_industrial,track_genre_iranian,track_genre_j-dance,track_genre_j-idol,track_genre_j-pop,track_genre_j-rock,track_genre_jazz,track_genre_k-pop,track_genre_kids,track_genre_latin,track_genre_latino,track_genre_malay,track_genre_mandopop,track_genre_metal,track_genre_metalcore,track_genre_minimal-techno,track_genre_mpb,track_genre_new-age,track_genre_opera,track_genre_pagode,track_genre_party,track_genre_piano,track_genre_pop,track_genre_pop-film,track_genre_power-pop,track_genre_progressive-house,track_genre_psych-rock,track_genre_punk,track_genre_punk-rock,track_genre_r-n-b,track_genre_reggae,track_genre_reggaeton,track_genre_rock,track_genre_rock-n-roll,track_genre_rockabilly,track_genre_romance,track_genre_sad,track_genre_salsa,track_genre_samba,track_genre_sertanejo,track_genre_show-tunes,track_genre_singer-songwriter,track_genre_ska,track_genre_sleep,track_genre_soul,track_genre_spanish,track_genre_study,track_genre_swedish,track_genre_synth-pop,track_genre_tango,track_genre_techno,track_genre_trance,track_genre_trip-hop,track_genre_turkish,track_genre_world-music
0,5SuOikwiRyPMVoIQDJUgSV,0.73,0.042473,0.0,0.686294,0.4610,0.090909,0.791392,0.0,0.148187,0.032329,0.000001,0.3580,0.718593,0.361245,0.8,Comedy,Gen Hoshino,Comedy,Comedy,Gen Hoshino,Comedy,0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,4qPNDBW1i3p13qLCt0Ki3A,0.55,0.026971,0.0,0.426396,0.1660,0.090909,0.597377,1.0,0.079067,0.927711,0.000006,0.1010,0.268342,0.318397,0.8,Ghost - Acoustic,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,Ben Woodward,Ghost (Acoustic),1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa

In [60]:
features

['Unnamed: 0',
 'track_id',
 'track_genre_acoustic',
 'track_genre_afrobeat',
 'track_genre_alt-rock',
 'track_genre_alternative',
 'track_genre_ambient',
 'track_genre_anime',
 'track_genre_black-metal',
 'track_genre_bluegrass',
 'track_genre_blues',
 'track_genre_brazil',
 'track_genre_breakbeat',
 'track_genre_british',
 'track_genre_cantopop',
 'track_genre_chicago-house',
 'track_genre_children',
 'track_genre_chill',
 'track_genre_classical',
 'track_genre_club',
 'track_genre_comedy',
 'track_genre_country',
 'track_genre_dance',
 'track_genre_dancehall',
 'track_genre_death-metal',
 'track_genre_deep-house',
 'track_genre_detroit-techno',
 'track_genre_disco',
 'track_genre_disney',
 'track_genre_drum-and-bass',
 'track_genre_dub',
 'track_genre_dubstep',
 'track_genre_edm',
 'track_genre_electro',
 'track_genre_electronic',
 'track_genre_emo',
 'track_genre_folk',
 'track_genre_forro',
 'track_genre_french',
 'track_genre_funk',
 'track_genre_garage',
 'track_genre_german',
 

In [61]:
# Check to see if song is in dataset
song= df[(df['artists'] == 'Kodak Black')]
song

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
51554,51554,2z718tUE8RwkPlJ3Oc2RcW,Kodak Black,Tic Tac - Just Rap,Love & War,0,239146,True,0.762,0.649,11,-5.624,0,0.0527,0.58000,0.000017,0.0971,0.5010,145.090,4,hip-hop
51556,51556,6XjFkmGaUioM800RmFSV8r,Kodak Black,"#Multiverse, If the Multiverse Is Real",Super Gremlin,4,200547,True,0.825,0.414,2,-6.634,1,0.1440,0.00265,0.000000,0.3570,0.1060,72.993,4,hip-hop
51558,51558,3cvVjjZU0IWuTsiaa5F7k3,Kodak Black,Pushin P - Top Rap,Love & War,3,239146,True,0.762,0.649,11,-5.624,0,0.0527,0.58000,0.000017,0.0971,0.5010,145.090,4,hip-hop
51560,51560,41vUI66skZegzFHxwMkk8t,Kodak Black,Walk - Fresh Rap Flow,Walk,0,166039,True,0.817,0.512,8,-6.943,1,0.0984,0.03700,0.000000,0.2160,0.1700,130.040,4,hip-hop
51566,51566,1qkjQxTfIyRtZPK1nYWTvw,Kodak Black,Sundown Rap,Remember the Times,1,203891,False,0.780,0.712,11,-3.624,0,0.0598,0.15500,0.000000,0.3340,0.0757,135.009,4,hip-hop
51568,51568,0EvlAy9hx3kbnqfgg8VX7z,Kodak Black,Hip Hop Slow Jamz,Skrt,0,224864,True,0.901,0.352,2,-10.038,1,0.0915,0.58500,0.000000,0.1220,0.1990,111.065,4,hip-hop
51569,51569,4JLeg2RgRkPg5Y4BLWwQjn,Kodak Black,Totally Rap,Super Gremlin,0,200547,True,0.825,0.414,2,-6.634,1,0.1440,0.00265,0.000000,0.3570,0.1060,72.993,4,hip-hop
51570,51570,5RgN2LdqCGnxztU1cTNJjf,Kodak Black,New Rap 20s,Super Gremlin,0,200547,True,0.825,0.414,2,-6.634,1,0.1440,0.00265,0.000000,0.3570,0.1060,72.993,4,hip-hop
51572,51572,19b4vDb9mrs4WFh42ySOi7,Kodak Black,Gaming Songs 2022: Hard Mode,Super Gremlin,0,200547,True,0.825,0.414,2,-6.634,1,0.1440,0.00265,0.000000,0.3570,0.1060,72.993,4,hip-hop
51573,51573,0osMWCvwxing4xMPbMEMJY,Kodak Black,New Grooves,Super Gremlin,0,200547,True,0.825,0.414,2,-6.634,1,0.1440,0.00265,0.000000,0.3570,0.1060,72.993,4,hip-hop


In [67]:
features

['Unnamed: 0',
 'track_id',
 'track_genre_acoustic',
 'track_genre_afrobeat',
 'track_genre_alt-rock',
 'track_genre_alternative',
 'track_genre_ambient',
 'track_genre_anime',
 'track_genre_black-metal',
 'track_genre_bluegrass',
 'track_genre_blues',
 'track_genre_brazil',
 'track_genre_breakbeat',
 'track_genre_british',
 'track_genre_cantopop',
 'track_genre_chicago-house',
 'track_genre_children',
 'track_genre_chill',
 'track_genre_classical',
 'track_genre_club',
 'track_genre_comedy',
 'track_genre_country',
 'track_genre_dance',
 'track_genre_dancehall',
 'track_genre_death-metal',
 'track_genre_deep-house',
 'track_genre_detroit-techno',
 'track_genre_disco',
 'track_genre_disney',
 'track_genre_drum-and-bass',
 'track_genre_dub',
 'track_genre_dubstep',
 'track_genre_edm',
 'track_genre_electro',
 'track_genre_electronic',
 'track_genre_emo',
 'track_genre_folk',
 'track_genre_forro',
 'track_genre_french',
 'track_genre_funk',
 'track_genre_garage',
 'track_genre_german',
 

In [70]:
features.remove('track_id')


['Unnamed: 0', 'track_genre_acoustic', 'track_genre_afrobeat', 'track_genre_alt-rock', 'track_genre_alternative', 'track_genre_ambient', 'track_genre_anime', 'track_genre_black-metal', 'track_genre_bluegrass', 'track_genre_blues', 'track_genre_brazil', 'track_genre_breakbeat', 'track_genre_british', 'track_genre_cantopop', 'track_genre_chicago-house', 'track_genre_children', 'track_genre_chill', 'track_genre_classical', 'track_genre_club', 'track_genre_comedy', 'track_genre_country', 'track_genre_dance', 'track_genre_dancehall', 'track_genre_death-metal', 'track_genre_deep-house', 'track_genre_detroit-techno', 'track_genre_disco', 'track_genre_disney', 'track_genre_drum-and-bass', 'track_genre_dub', 'track_genre_dubstep', 'track_genre_edm', 'track_genre_electro', 'track_genre_electronic', 'track_genre_emo', 'track_genre_folk', 'track_genre_forro', 'track_genre_french', 'track_genre_funk', 'track_genre_garage', 'track_genre_german', 'track_genre_gospel', 'track_genre_goth', 'track_genre

In [74]:
df_final['track_name'] = df_final["track_name_x"]
df_final['artists'] = df_final["artists_x"]
df_final['album_name'] = df_final["album_name_x"]

In [75]:
from sklearn.neighbors import NearestNeighbors

# define the number of nearest neighbors to consider
k = 6

# define a function to recommend songs based on a given song name
def recommend_song(track_name, df_final, features):
    
    # initialize the model with the number of neighbors
    model = NearestNeighbors(n_neighbors=k)

    # fit the model to the data
    model.fit(df_final[features])
    
    # get the track_id of the given track name
    track_id = df_final[df_final['track_name'] == track_name]['track_id'].iloc[0]
    
    # get the index of the tracks in the model dataframe
    idx = df_final[df_final['track_id'] == track_id].index[0]
    
    # get the features of the tracks
    track_features = df_final.loc[idx, features].values.reshape(1, -1)
    
    # find the k nearest neighbors
    distances, indices = model.kneighbors(track_features)
    
    # get the track names of the nearest neighbors
    tracks = df_final.iloc[indices[0]]
    tracks["distance"] = distances[0]
    
    return tracks

In [81]:
recommend_song('2055', df_final, features)

,track_id,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_name_x,artists_x,album_name_x,track_name_y,artists_y,album_name_y,Unnamed: 0,track_genre_acoustic,track_genre_afrobeat,track_genre_alt-rock,track_genre_alternative,track_genre_ambient,track_genre_anime,track_genre_black-metal,track_genre_bluegrass,track_genre_blues,track_genre_brazil,track_genre_breakbeat,track_genre_british,track_genre_cantopop,track_genre_chicago-house,track_genre_children,track_genre_chill,track_genre_classical,track_genre_club,track_genre_comedy,track_genre_country,track_genre_dance,track_genre_dancehall,track_genre_death-metal,track_genre_deep-house,track_genre_detroit-techno,track_genre_disco,track_genre_disney,track_genre_drum-and-bass,track_genre_dub,track_genre_dubstep,track_genre_edm,track_genre_electro,track_genre_electronic,track_genre_emo,track_genre_folk,track_genre_forro,track_genre_french,track_genre_funk,track_genre_garage,track_genre_german,track_genre_gospel,track_genre_goth,track_genre_grindcore,track_genre_groove,track_genre_grunge,track_genre_guitar,track_genre_happy,track_genre_hard-rock,track_genre_hardcore,track_genre_hardstyle,track_genre_heavy-metal,track_genre_hip-hop,track_genre_honky-tonk,track_genre_house,track_genre_idm,track_genre_indian,track_genre_indie,track_genre_indie-pop,track_genre_industrial,track_genre_iranian,track_genre_j-dance,track_genre_j-idol,track_genre_j-pop,track_genre_j-rock,track_genre_jazz,track_genre_k-pop,track_genre_kids,track_genre_latin,track_genre_latino,track_genre_malay,track_genre_mandopop,track_genre_metal,track_genre_metalcore,track_genre_minimal-techno,track_genre_mpb,track_genre_new-age,track_genre_opera,track_genre_pagode,track_genre_party,track_genre_piano,track_genre_pop,track_genre_pop-film,track_genre_power-pop,track_genre_progressive-house,track_genre_psych-rock,track_genre_punk,track_genre_punk-rock,track_genre_r-n-b,track_genre_reggae,track_genre_reggaeton,track_genre_rock,track_genre_rock-n-roll,track_genre_rockabilly,track_genre_romance,track_genre_sad,track_genre_salsa,track_genre_samba,track_genre_sertanejo,track_genre_show-tunes,track_genre_singer-songwriter,track_genre_ska,track_genre_sleep,track_genre_soul,track_genre_spanish,track_genre_study,track_genre_swedish,track_genre_synth-pop,track_genre_tango,track_genre_techno,track_genre_trance,track_genre_trip-hop,track_genre_turkish,track_genre_world-music,track_name,artists,album_name,distance
71211,4XvcHTUfIlWfyJTRG0aqlo,0.81,0.022070,1.0,0.841624,0.512,0.545455,0.807687,0.0,0.193782,0.493976,0.000000,0.120,0.641206,0.330815,0.8,2055,Sleepy Hallow,Still Sleep?,2055,Sleepy Hallow,Still Sleep?,87008,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2055,Sleepy Hallow,Still Sleep?,3.330669e-16
71238,0mG6igzxbH55XRyq3ygEh8,0.73,0.022070,1.0,0.786802,0.518,0.545455,0.807835,0.0,0.322280,0.456827,0.000000,0.121,0.649246,0.661584,0.8,2055,Sleepy Hallow,2055,2055,Sleepy Hallow,2055,87100,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

In [77]:
# define the number of nearest neighbors to consider
k = 6

# define a function to recommend songs based on a given song name
def recommend_song_artist(track_name, artist, df_final, features):
    
    train = df_final.loc[df_final.artists == artist]
    
    k = min(len(train), 6)
    
    # initialize the model with the number of neighbors
    model = NearestNeighbors(n_neighbors=k)

    # fit the model to the data
    model.fit(train[features])
    
    # get the track_id of the given track name
    track_id = df_final[df_final['track_name'] == track_name]['track_id'].iloc[0]
    
    # get the index of the tracks in the model dataframe
    idx = df_final[df_final['track_id'] == track_id].index[0]
    
    # get the features of the tracks
    track_features = df_final.loc[idx, features].values.reshape(1, -1)
    
    # find the k nearest neighbors
    distances, indices = model.kneighbors(track_features)
    
    # get the track names of the nearest neighbors
    tracks = train.iloc[indices[0]]
    tracks["distance"] = distances[0]
    
    return tracks

In [78]:
recommend_song_artist('You & I (Nobody in the World)', 'John Legend', df_final, features)

,track_id,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_name_x,artists_x,album_name_x,track_name_y,artists_y,album_name_y,Unnamed: 0,track_genre_acoustic,track_genre_afrobeat,track_genre_alt-rock,track_genre_alternative,track_genre_ambient,track_genre_anime,track_genre_black-metal,track_genre_bluegrass,track_genre_blues,track_genre_brazil,track_genre_breakbeat,track_genre_british,track_genre_cantopop,track_genre_chicago-house,track_genre_children,track_genre_chill,track_genre_classical,track_genre_club,track_genre_comedy,track_genre_country,track_genre_dance,track_genre_dancehall,track_genre_death-metal,track_genre_deep-house,track_genre_detroit-techno,track_genre_disco,track_genre_disney,track_genre_drum-and-bass,track_genre_dub,track_genre_dubstep,track_genre_edm,track_genre_electro,track_genre_electronic,track_genre_emo,track_genre_folk,track_genre_forro,track_genre_french,track_genre_funk,track_genre_garage,track_genre_german,track_genre_gospel,track_genre_goth,track_genre_grindcore,track_genre_groove,track_genre_grunge,track_genre_guitar,track_genre_happy,track_genre_hard-rock,track_genre_hardcore,track_genre_hardstyle,track_genre_heavy-metal,track_genre_hip-hop,track_genre_honky-tonk,track_genre_house,track_genre_idm,track_genre_indian,track_genre_indie,track_genre_indie-pop,track_genre_industrial,track_genre_iranian,track_genre_j-dance,track_genre_j-idol,track_genre_j-pop,track_genre_j-rock,track_genre_jazz,track_genre_k-pop,track_genre_kids,track_genre_latin,track_genre_latino,track_genre_malay,track_genre_mandopop,track_genre_metal,track_genre_metalcore,track_genre_minimal-techno,track_genre_mpb,track_genre_new-age,track_genre_opera,track_genre_pagode,track_genre_party,track_genre_piano,track_genre_pop,track_genre_pop-film,track_genre_power-pop,track_genre_progressive-house,track_genre_psych-rock,track_genre_punk,track_genre_punk-rock,track_genre_r-n-b,track_genre_reggae,track_genre_reggaeton,track_genre_rock,track_genre_rock-n-roll,track_genre_rockabilly,track_genre_romance,track_genre_sad,track_genre_salsa,track_genre_samba,track_genre_sertanejo,track_genre_show-tunes,track_genre_singer-songwriter,track_genre_ska,track_genre_sleep,track_genre_soul,track_genre_spanish,track_genre_study,track_genre_swedish,track_genre_synth-pop,track_genre_tango,track_genre_techno,track_genre_trance,track_genre_trip-hop,track_genre_turkish,track_genre_world-music,track_name,artists,album_name,distance
81329,55nlbqqFVnSsArIeYSQlqx,0.63,0.046678,0.0,0.462944,0.342,1.000000,0.765200,0.0,0.029326,0.129518,0.0,0.1240,0.225126,0.345508,0.8,You & I (Nobody in the World),John Legend,Love In The Future (Expanded Edition),You & I (Nobody in the World),John Legend,Love In The Future (Expanded Edition),103926,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,You & I (Nobody in the World),John Legend,Love In The Future (Expanded Edition),0.000000
81239,0QjQC8sGqxUkaJFBsJjtGm,0.60,0.043427,0.0,0.366497,0.357,0.727273,0.760372,0.0,0.032332,0.706827,0.0,0.2190,0.153769,0.505243,0.8,Conversations in the Dark,John Legend,Bigger Love,Conversations in the Dark,John Legend,Bigger Love,103714,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

In [84]:
# df_final.to_csv('df_final.csv', index=False)


In [79]:
from sklearn.neighbors import NearestNeighbors

# define the number of nearest neighbors to consider
k = 6

# define a function to recommend songs based on a given song name
def recommend_song(track_name, df_final, features):
    
    # initialize the model with the number of neighbors
    model = NearestNeighbors(n_neighbors=k, metric='cosine')

    # fit the model to the data
    model.fit(df_final[features])
    
    # get the track_id of the given track name
    track_id = df_final[df_final['track_name'] == track_name]['track_id'].iloc[0]
    
    # get the index of the tracks in the model dataframe
    idx = df_final[df_final['track_id'] == track_id].index[0]
    
    # get the features of the tracks
    track_features = df_final.loc[idx, features].values.reshape(1, -1)
    
    # find the k nearest neighbors
    distances, indices = model.kneighbors(track_features)
    
    # get the track names of the nearest neighbors
    tracks = df_final.iloc[indices[0]]
    tracks["distance"] = distances[0]
    
    return tracks

In [82]:
recommend_song('2055', df_final, features)

,track_id,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_name_x,artists_x,album_name_x,track_name_y,artists_y,album_name_y,Unnamed: 0,track_genre_acoustic,track_genre_afrobeat,track_genre_alt-rock,track_genre_alternative,track_genre_ambient,track_genre_anime,track_genre_black-metal,track_genre_bluegrass,track_genre_blues,track_genre_brazil,track_genre_breakbeat,track_genre_british,track_genre_cantopop,track_genre_chicago-house,track_genre_children,track_genre_chill,track_genre_classical,track_genre_club,track_genre_comedy,track_genre_country,track_genre_dance,track_genre_dancehall,track_genre_death-metal,track_genre_deep-house,track_genre_detroit-techno,track_genre_disco,track_genre_disney,track_genre_drum-and-bass,track_genre_dub,track_genre_dubstep,track_genre_edm,track_genre_electro,track_genre_electronic,track_genre_emo,track_genre_folk,track_genre_forro,track_genre_french,track_genre_funk,track_genre_garage,track_genre_german,track_genre_gospel,track_genre_goth,track_genre_grindcore,track_genre_groove,track_genre_grunge,track_genre_guitar,track_genre_happy,track_genre_hard-rock,track_genre_hardcore,track_genre_hardstyle,track_genre_heavy-metal,track_genre_hip-hop,track_genre_honky-tonk,track_genre_house,track_genre_idm,track_genre_indian,track_genre_indie,track_genre_indie-pop,track_genre_industrial,track_genre_iranian,track_genre_j-dance,track_genre_j-idol,track_genre_j-pop,track_genre_j-rock,track_genre_jazz,track_genre_k-pop,track_genre_kids,track_genre_latin,track_genre_latino,track_genre_malay,track_genre_mandopop,track_genre_metal,track_genre_metalcore,track_genre_minimal-techno,track_genre_mpb,track_genre_new-age,track_genre_opera,track_genre_pagode,track_genre_party,track_genre_piano,track_genre_pop,track_genre_pop-film,track_genre_power-pop,track_genre_progressive-house,track_genre_psych-rock,track_genre_punk,track_genre_punk-rock,track_genre_r-n-b,track_genre_reggae,track_genre_reggaeton,track_genre_rock,track_genre_rock-n-roll,track_genre_rockabilly,track_genre_romance,track_genre_sad,track_genre_salsa,track_genre_samba,track_genre_sertanejo,track_genre_show-tunes,track_genre_singer-songwriter,track_genre_ska,track_genre_sleep,track_genre_soul,track_genre_spanish,track_genre_study,track_genre_swedish,track_genre_synth-pop,track_genre_tango,track_genre_techno,track_genre_trance,track_genre_trip-hop,track_genre_turkish,track_genre_world-music,track_name,artists,album_name,distance
71211,4XvcHTUfIlWfyJTRG0aqlo,0.81,0.022070,1.0,0.841624,0.512,0.545455,0.807687,0.0,0.193782,0.493976,0.000000,0.120,0.641206,0.330815,0.8,2055,Sleepy Hallow,Still Sleep?,2055,Sleepy Hallow,Still Sleep?,87008,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2055,Sleepy Hallow,Still Sleep?,3.330669e-16
71238,0mG6igzxbH55XRyq3ygEh8,0.73,0.022070,1.0,0.786802,0.518,0.545455,0.807835,0.0,0.322280,0.456827,0.000000,0.121,0.649246,0.661584,0.8,2055,Sleepy Hallow,2055,2055,Sleepy Hallow,2055,87100,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False